<a href="https://colab.research.google.com/github/apriandito/dkem/blob/main/Geospasial_DEKM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import folium
import requests
from branca.colormap import LinearColormap
import json
from io import StringIO

In [ ]:
# Load poverty data
poverty_data = pd.read_csv("https://raw.githubusercontent.com/yogasgm/poverty_data/main/persentase_kemiskinan_cleaned.csv")

# Load GeoJSON Indonesia Province
geojson_url = "https://raw.githubusercontent.com/superpikar/indonesia-geojson/master/indonesia-province.json"
try:
    response = requests.get(geojson_url)
    response.raise_for_status()
    geojson_data = response.json()
except (requests.RequestException, json.JSONDecodeError) as e:
    print(f"Failed to fetch or parse GeoJSON data from URL: {e}")
    print("Using fallback GeoJSON data...")

In [ ]:
# Function to normalize province names
def normalize_province_name(name):
    replacements = {
        'DI. ACEH': 'ACEH',
        'DAERAH ISTIMEWA YOGYAKARTA': 'DI YOGYAKARTA',
        'PROBANTEN': 'BANTEN',
        'BANGKA BELITUNG': 'KEP, BANGKA BELITUNG',
        'NUSATENGGARA BARAT': 'NUSA TENGGARA BARAT',
        'IRIAN JAYA BARAT': 'PAPUA BARAT',
        'IRIAN JAYA TIMUR': 'PAPUA',
        'IRIAN JAYA TENGAH': 'PAPUA TENGAH',
        'RIAU': 'KEP, RIAU'
    }
    name = name.upper().strip()
    return replacements.get(name, name)

In [ ]:
# Clean the poverty data
poverty_data['persentase'] = poverty_data['persentase'].astype(float)

# Normalize province names in poverty data
poverty_data['provinsi'] = poverty_data['provinsi'].apply(normalize_province_name)
poverty_dict = dict(zip(poverty_data['provinsi'], poverty_data['persentase']))

In [ ]:
# Function to get province name from feature
def get_province_name(feature):
    return normalize_province_name(feature['properties']['Propinsi'])

# Function to get poverty percentage from feature
def get_poverty_percentage(feature):
    province_name = get_province_name(feature)
    return poverty_dict.get(province_name, "Data tidak tersedia")

In [ ]:
# Add poverty percentage to each features
for feature in geojson_data['features']:
    feature['properties']['poverty_percentage'] = f"{get_poverty_percentage(feature)}%"

In [ ]:
# Create a map centered on Indonesia
m = folium.Map(location=[-2.5, 118], zoom_start=5)

# Create a color map
color_map = LinearColormap(colors=['white', 'yellow', 'orange', 'red'],
                           vmin=poverty_data['persentase'].min(),
                           vmax=poverty_data['persentase'].max())

# Add the colored polygons to the map
folium.GeoJson(
    geojson_data,
    style_function=lambda feature: {
        'fillColor': color_map(poverty_dict.get(get_province_name(feature), 0)),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['Propinsi', 'poverty_percentage'],
        aliases=['Provinsi:', 'Persentase Kemiskinan:'],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    ),
    popup=folium.GeoJsonPopup(
        fields=['Propinsi', 'poverty_percentage'],
        aliases=['Provinsi:', 'Persentase Kemiskinan:'],
        localize=True,
        labels=True,
    )
).add_to(m)

# Add a color legend
color_map.add_to(m)

# Save the map
m.save('indonesia_poverty_map_simple.html')

print("Map has been generated and saved as 'indonesia_poverty_map_simple.html'")

Map has been generated and saved as 'indonesia_poverty_map_simple.html'


In [ ]:
# Print provinces that were not matched
geojson_provinces = set(get_province_name(feature) for feature in geojson_data['features'])
csv_provinces = set(poverty_data['provinsi'])
print("Provinces in GeoJSON but not in CSV:", geojson_provinces - csv_provinces)
print("Provinces in CSV but not in GeoJSON:", csv_provinces - geojson_provinces)

Provinces in GeoJSON but not in CSV: set()
Provinces in CSV but not in GeoJSON: {'PAPUA SELATAN', 'KALIMANTAN UTARA', 'PAPUA PEGUNUNGAN', 'SULAWESI BARAT', 'PAPUA BARAT DAYA'}
